In [ ]:
import os
import pandas as pd
import numpy as np
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import matplotlib.pyplot as plt
import seaborn as sns
import shap

In [ ]:
csv_dir = '../../SEC Trackman Data/'
all_files = [os.path.join(csv_dir, f) for f in os.listdir(csv_dir) if f.endswith('.csv')]

df_list = []

for file in all_files:
    df = pd.read_csv(file)
    df_list.append(df)

all_pitches = pd.concat(df_list, ignore_index=True)

print(all_pitches.shape)
print(all_pitches.columns)

In [ ]:
fs = all_pitches[all_pitches['TaggedPitchType'].isin(['Sinker', 'TwoSeamFastBall'])]
print(fs.shape)

In [ ]:
fs['whiff'] = np.where(fs['PitchCall']=='StrikeSwinging',1,0)

In [ ]:
preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']

In [ ]:
fs_X = fs[preds]
fs_y = fs['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(fs_X, fs_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
aub_fs = fs[fs['PitcherTeam']=='AUB_TIG']

In [ ]:
# Get unique values
unique_values = fs['Pitcher'].unique()
aub_unique_values = aub_fs['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: fs[fs['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: fs[fs['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_fs_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(fs_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} Sinker stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_fs_stuff(pitcher)

In [ ]:
aub_subsets['Cannon, Will']

In [ ]:
fs_rr = fs[(fs['PitcherThrows'] == 'Right') & (fs['BatterSide'] == 'Right')]
fs_rl = fs[(fs['PitcherThrows'] == 'Right') & (fs['BatterSide'] == 'Left')]
fs_lr = fs[(fs['PitcherThrows'] == 'Left') & (fs['BatterSide'] == 'Right')]
fs_ll = fs[(fs['PitcherThrows'] == 'Left') & (fs['BatterSide'] == 'Left')]

print(fs_rr.shape)
print(fs_rl.shape)
print(fs_lr.shape)
print(fs_ll.shape)

In [ ]:
fs_rr['whiff'] = np.where(fs_rr['PitchCall']=='StrikeSwinging',1,0)
fs_rl['whiff'] = np.where(fs_rl['PitchCall']=='StrikeSwinging',1,0)
fs_lr['whiff'] = np.where(fs_lr['PitchCall']=='StrikeSwinging',1,0)
fs_ll['whiff'] = np.where(fs_ll['PitchCall']=='StrikeSwinging',1,0)

In [ ]:
preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']

In [ ]:
fs_rr_X = fs_rr[preds]
fs_rr_y = fs_rr['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(fs_rr_X, fs_rr_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
aub_fs_rr = fs_rr[fs_rr['PitcherTeam']=='AUB_TIG']
print(aub_fs_rr.shape)

In [ ]:
# Get unique values
unique_values = fs_rr['Pitcher'].unique()
aub_unique_values = aub_fs_rr['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: fs_rr[fs_rr['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: fs_rr[fs_rr['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_rr_fs_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(fs_rr_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} RR Sinker stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_rr_fs_stuff(pitcher)

In [ ]:
fs_rl_X = fs_rl[preds]
fs_rl_y = fs_rl['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(fs_rl_X, fs_rl_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
aub_fs_rl = fs_rl[fs_rl['PitcherTeam']=='AUB_TIG']
print(aub_fs_rl.shape)

In [ ]:
# Get unique values
unique_values = fs_rl['Pitcher'].unique()
aub_unique_values = aub_fs_rl['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: fs_rl[fs_rl['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: fs_rl[fs_rl['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_rl_fs_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(fs_rl_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} RL Sinker stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_rl_fs_stuff(pitcher)

In [ ]:
aub_subsets['Cannon, Will']

In [ ]:
fs_lr_X = fs_lr[preds]
fs_lr_y = fs_lr['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(fs_lr_X, fs_lr_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
aub_fs_lr = fs_lr[fs_lr['PitcherTeam']=='AUB_TIG']
print(aub_fs_lr.shape)

In [ ]:
# Get unique values
unique_values = fs_lr['Pitcher'].unique()
aub_unique_values = aub_fs_lr['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: fs_lr[fs_lr['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: fs_lr[fs_lr['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_lr_fs_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(fs_lr_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} LR Sinker stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_lr_fs_stuff(pitcher)

In [ ]:
fs_ll_X = fs_ll[preds]
fs_ll_y = fs_ll['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(fs_ll_X, fs_ll_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
aub_fs_ll = fs_ll[fs_ll['PitcherTeam']=='AUB_TIG']
print(aub_fs_ll.shape)

In [ ]:
# Get unique values
unique_values = fs_ll['Pitcher'].unique()
aub_unique_values = aub_fs_ll['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: fs_ll[fs_ll['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: fs_ll[fs_ll['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_ll_fs_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(fs_ll_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} LL Sinker stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_ll_fs_stuff(pitcher)

In [ ]:
aub_subsets['Keplinger, Konner']